In [56]:
import os
import pandas as pd
import requests

In [57]:
league = os.getenv('LEAGUE')
def roster(league):
    request = requests.get(f'https://api.sleeper.app/v1/league/{league}/rosters')
    return request.json()

In [58]:
def users(league):
    request = requests.get(f'https://api.sleeper.app/v1/league/{league}/users')
    return request.json()

In [59]:
json = roster(league)
roster= pd.DataFrame(json)
json = users(league)
user = pd.DataFrame(json)

In [66]:
def users_transform(data1, data2):

    #Using data from the users api
    df = pd.DataFrame(data1)
    df = df[['user_id', 'metadata', 'display_name']]
    df['team_name'] = [met['team_name'] for met in df['metadata']]

    #Using the data from the roster api
    df2 = pd.DataFrame(data2)
    settings_df = pd.json_normalize(df2['settings'])
    df2 = pd.concat([df2.drop(columns=['settings']), settings_df], axis=1)

    #Concating the two dataframes
    df = pd.concat([df, df2[['total_moves', 'wins', 'ties', 'losses', 'waiver_position', 'fpts']]])

    return df

In [67]:
df = users_transform(user, roster)